In [21]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers import Adam

In [3]:
embed=hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [5]:
embed(['friends'])

<tf.Tensor: shape=(1, 250), dtype=float32, numpy=
array([[-8.2660869e-02, -1.7848998e-02, -1.0665781e-02,  4.0200830e-02,
        -9.8001614e-02,  8.2352221e-02,  1.0365940e-03,  4.6690527e-02,
         3.6734596e-02, -5.8365002e-02,  1.4857404e-02,  5.1676743e-02,
         5.1263995e-02,  3.0565221e-02, -4.2199274e-03, -1.0558112e-01,
         3.5203446e-02,  1.8931816e-04, -5.5870552e-02, -2.2511218e-02,
         5.0791752e-02,  2.0086231e-02,  7.3669173e-02,  3.5827946e-02,
         5.6499965e-03, -2.7705513e-02,  8.8337675e-02,  6.9989979e-02,
        -2.7089190e-02,  1.2496362e-01,  1.2648205e-02,  3.3584066e-02,
        -1.8621382e-03, -3.8032766e-02,  3.7724204e-02,  1.6809585e-02,
        -1.2898981e-02,  1.2022736e-01,  9.4549716e-02,  3.0786434e-02,
         2.6737809e-02, -1.8429162e-01,  1.9299038e-02, -3.6845256e-02,
         5.3915819e-03, -1.2482375e-02, -1.0086684e-01,  1.8922809e-01,
        -4.9780525e-02, -7.2227843e-02,  1.0012158e-03, -1.0895739e-02,
         8.061

In [7]:
def get_max_length(df):
  max_length=0
  for row in df['review']:
    text=row.split()
    if len(text)> max_length:
      max_length=len(text)
  return max_length

In [19]:
def get_encoded_review(reviews):
  encoded_reviews=[]
  for review in reviews:
    token=review.split()
    encode_text=embed(token)
    encoded_reviews.append(encode_text)
  return encoded_reviews


In [23]:
def pad_encoded_reviews(encoded_reviews,max_length):
  pad_encoded=[]
  for enc_review in encoded_reviews:
    zero_pad_cnt=max_length-enc_review.shape[0]
    pad=np.zeros((1,250))
    for i in range(zero_pad_cnt):
      enc_review=np.concatenate((pad,enc_review),axis=0)
    pad_encoded.append(enc_review)
  return pad_encoded

In [9]:
def sentiment_encode(sentiment):
  if sentiment == 'positive':
    return[1,0]
  else:
    return[0,1]

In [10]:
def preprocess(df,max_length):
  reviews=df['review'].tolist()
  encoded_reviews=get_encoded_review(reviews)
  pad_sequence=pad_encoded_reviews(encoded_reviews,max_length)
  sentiment_list=df['sentiment'].tolist()
  sentiment_encoded=[sentiment_encode(sentiment) for sentiment in sentiment_list]
  x=np.array(pad_sequence)
  y=np.array(sentiment_encoded)
  return x,y

In [11]:
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]

In [12]:
df=pd.DataFrame(movie_reviews_train)
df

,review,sentiment
0,this is the best movie,positive
1,i recommend you watch this movie,positive
2,it was waste of money and time,negative
3,the worst movie ever,negative


In [24]:
max_length=get_max_length(df)
print("lenngthiest sequence is------",max_length)
x_train,y_train=preprocess(df,max_length)
print(x_train.shape)

lenngthiest sequence is------ 7
(4, 7, 250)


In [25]:
print(x_train[0])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.04606383 -0.08771229  0.07026413 ...  0.00152413  0.06032765
  -0.02329596]
 ...
 [-0.03488984 -0.12522927  0.01865043 ... -0.01779286  0.03092172
  -0.01291556]
 [ 0.00653369 -0.01256914 -0.00868887 ... -0.0331819   0.08193181
   0.00817045]
 [-0.04702299  0.01101733 -0.03464236 ...  0.0377473  -0.00763348
   0.07064962]]


In [33]:
model=Sequential()
model.add(LSTM(32))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 0.6967 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6856 - accuracy: 0.7500
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6747 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6638 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6529 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6420 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6308 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6194 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6077 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 0.5956 - accuracy: 1.0000
Epoch 11/50
1/1 [=======================

In [34]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 32)                36224     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


In [35]:
movie_reviews_test = [
         {'review': 'it is better movie', 'sentiment': 'positive'},
         {'review': 'i suggest you see this movie', 'sentiment': 'positive'},
         {'review': 'it was just throwing 20 dollars away', 'sentiment': 'negative'},
         {'review': 'worse than any show', 'sentiment': 'negative'},
         {'review': 'nice movie, so love it', 'sentiment': 'positive'},
         {'review': 'terrible', 'sentiment': 'negative'}
    ]


In [37]:
test_df = pd.DataFrame(movie_reviews_test)

test_X, test_Y = preprocess(test_df,max_length)



In [38]:
score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 0s - loss: 0.1727 - accuracy: 1.0000
Test score: 0.17266005277633667
Test accuracy: 1.0
